# YOLO Step 3 - Final Run on Test Data

*withOUT Synthetic data*

Roboflow data generated:
* Final Version - no augmentation
* Version 15


In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import io
import pandas as pd
import json
import os
import glob 


In [2]:
roboflow_version = 15
parent_dir = "/home/ec2-user/AutoArki/yolov5/yolov5/autoarki-"

In [3]:
#!pip install roboflow
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5")

from roboflow import Roboflow
rf = Roboflow(api_key="pvpFbwxA6FFS6pAwYdNk")
project = rf.workspace("new-workspace-s32a1").project("autoarki")
dataset = project.version(roboflow_version).download("yolov5")

roboflow_version_str = str(roboflow_version)+"/"

loading Roboflow workspace...
loading Roboflow project...

IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



Extracting Dataset Version Zip to autoarki-15 in yolov5pytorch:: 100%|██████████| 5198/5198 [00:00<00:00, 12331.44it/s]


In [4]:
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5")

!rm -R train; mkdir train
!rm -R test; mkdir test
!rm -R valid; mkdir valid
!cp {parent_dir+roboflow_version_str}train/images/*  train --recursive
!cp {parent_dir+roboflow_version_str}train/labels/*  train --recursive
!cp {parent_dir+roboflow_version_str}test/images/*  test --recursive
!cp {parent_dir+roboflow_version_str}test/labels/*  test --recursive
!cp {parent_dir+roboflow_version_str}valid/images/*  valid --recursive
!cp {parent_dir+roboflow_version_str}valid/labels/*  valid --recursive



# Check 

In [5]:
def count_classes(file_path):
    '''
    increment class_counts dictionary for each instance of a 
    class check against roboflow summary (health check)
    
    '''
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_counts[og_line[0]] +=1
    return class_counts

**Class Order**  <br>
0: Column_Wall_Intersection <br>
1: cmu_hinge_clearance <br>
2: floating_door <br>
3: illegibletext <br>
4: missing_wall <br>
5: no_stair_direction

In [6]:
#Count of class types in train
class_counts = {'0':0, '1':0, '2':0, '3': 0, '4': 0, '5': 0, '6':0, '7':0}

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/train")
for file in glob.glob("*.txt"):
    count_classes(file)
print(class_counts)

{'0': 248, '1': 20, '2': 611, '3': 5437, '4': 132, '5': 484, '6': 0, '7': 0}


# Replace Class Type

Drop -, cmu, and missing_wall classes

In [7]:
classes = {'0': '0', # Column Wall intersection
           '2': '1', # Floating Door
           '3': '2', # Illegible Text
           '5': '3'  # No stair direction
          } 

classes_to_omit = ['1','4']

In [8]:
def replace_class_n_with_class_name(file_path):
    '''for a txt file replace the class number with a class name'''
    new_lines = []
    infile = open(file_path, "r")
    lines = infile.readlines()

    if len(lines) == 0:
        pass
    else:
        for og_line in lines:
            class_n = og_line[0]
            if class_n in classes_to_omit: 
                continue
            class_str = classes[class_n]
            new_line = og_line.replace(class_n, class_str, 1)
            new_lines.append(new_line)

    with open(file_path, 'w') as outfile:
        outfile.write(''.join(new_lines))

In [9]:
# replace class names for train and test txt files

os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/train")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)
    
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5/test")
for file in glob.glob("*.txt"):
    replace_class_n_with_class_name(file)    
    

# Install W&B login to track model - only do once

In [ ]:
!pip install wandb
!wandb login

Defaulting to user installation because normal site-packages is not writeable
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
Aborted!


In [40]:
import wandb

wandb.init(project="my-test-project", entity="kbaumstan")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /home/ec2-user/.netrc
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


# Learning

use smallest YOLO model to start yolov5s.pt

In [10]:
import torch
import torchvision

'0' # Column Wall intersection <br>
'1', # Floating Door <br>
'2', # Illegible Text <br>
'3'  # No stair direction <br>


## 300 Epochs run on best model weights from last run 
[link to last run](https://wandb.ai/auto-arki/custom_yolov5/artifacts/model/run_1m9h8hg3_model/45341bfe5569c1a21832)

In [11]:
os.chdir("/home/ec2-user/AutoArki/yolov5/yolov5")
!python train.py\
    --data custom_dataset-final.yaml \
    --epochs 300 \
    --project custom_yolov5 \
    --bbox_interval 1 \
    --save-period 10 \
    #--img 1280 \
    --patience 10 \
    --weights yolov5x.pt \
    --optimizer 'AdamW' \
    --cos-lr false \
    --batch-size 10


wandb: Currently logged in as: auto-arki (use `wandb login --relogin` to force relogin)
train: weights=yolov5s.pt, cfg=, data=custom_dataset-final.yaml, hyp=data/hyps/hyp.scratch-low.yaml, epochs=300, batch_size=16, imgsz=640, rect=False, resume=False, nosave=False, noval=False, noautoanchor=False, evolve=None, bucket=, cache=None, image_weights=False, device=, multi_scale=False, single_cls=False, optimizer=SGD, sync_bn=False, workers=8, project=custom_yolov5, name=exp, exist_ok=False, quad=False, cos_lr=False, label_smoothing=0.0, patience=100, freeze=[0], save_period=10, local_rank=-1, entity=None, upload_dataset=False, bbox_interval=1, artifact_alias=latest
remote: Enumerating objects: 1661, done.
remote: Counting objects: 100% (716/716), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 1661 (delta 714), reused 704 (delta 704), pack-reused 945
Receiving objects: 100% (1661/1661), 516.84 KiB | 15.20 MiB/s, done.
Resolving deltas: 100% (1219/1219), completed with 1